# Árboles de decisión 

Vamos a utilizar graphviz para visualización de los árboles.

Les dejo un [tutorial](https://bobswift.atlassian.net/wiki/spaces/GVIZ/pages/20971549/How+to+install+Graphviz+software) que ayuda en la instalación, especialmente para Windows y Mac OS.

In [ ]:
import graphviz
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import datasets
from sklearn import tree
from matplotlib.colors import ListedColormap

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

## Introducción a los árboles de decisión

In [ ]:
data = pd.DataFrame({"Dientes":[True,True,True,False,True,True,True,True,True,False],
                     "Pelo":[True,True,False,True,True,True,False,False,True,False],
                     "Respira":[True,True,True,True,True,True,False,True,True,True],
                     "Piernas":[True,True,False,True,True,True,False,False,True,True],
                     "Especie":["Mamifero","Mamifero","Reptil","Mamifero","Mamifero","Mamifero","Reptil","Reptil","Mamifero","Reptil"]
                     }, 
                    columns=["Dientes","Pelo","Respira","Piernas","Especie"]
                    )

features = data[["Dientes","Pelo","Respira","Piernas"]]
target = data["Especie"]

data

Vemos que la base de datos cuenta con 10 animales, sobre los cuales se ha observado si presentan dientes, si tienen pelo, si respiran y si tienen piernas. La variable dependiente en este caso es la especie. Se desea saber si, a partir de las variables registradas, se puede generar un clasificador que determine si un animal es mamífero o reptil. Para esto utilizaremos un árbol de decisión.

## Implementación

La implementación en python es sencilla. Primero, lo implementaremos con scikit-learn y luego iremos entendiendo qué está haciendo el método de este paquete.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion = 'entropy')

model.fit(features,target)

print("The prediction accuracy is: ",model.score(features,target)*100,"%")
#prediction = model.predict(test_features)

In [ ]:
plt.figure(figsize=(15,7))
#tree.plot_tree?
tree.plot_tree(model,feature_names=["Dientes","Pelo","Respira","Piernas"], filled=True)
plt.show()

## Ahora entendamos un poco


Entropía $H$:

Medida de impureza de un conjunto de datos. La ganancia de información se refleja en un decrecimiento en la medida de entropía.

Definición: 

$H(Q_m) = - \sum_{k \in target} P_{mk} log_2(P_{mk})$,

donde $Q_m$ son los datos en el nodo $m$ del árbol, la suma se extiende sobre los posibles valores $k$ de la variable respuesta y $P_{mk}$ es la probabilidad condicional que la variable respuesta tome el valor $k$ dado a que estamos en el nodo $m$

Caso Gini: $H(Q_m) = \sum_{k \in target} P_{mk} (1-P_{mk}))$

[Documentación](https://scikit-learn.org/stable/modules/tree.html#tree-mathematical-formulation)

In [ ]:
def entropia(P):
    '''
    Función que calcula medida de entropía dada una probabilidad P.
    '''
    entropy = - P * np.log2(P)
    return entropy

Para utilizar la definición de entropía debemos tener una función que estime la probabilidad con la frecuencia relativa.  

In [ ]:
def calc_prob(data):
    '''
    Función que calcula la probabilidad de cada clase.
    data : columna categórica de un dataframe de pandas.
    '''
    return data.value_counts()/data.shape[0]
probabilidades = calc_prob(data['Especie'])
probabilidades

Con esta función, podemos calcular la entropía total de los datos en el primer nodo:

In [ ]:
entropia_total = np.sum([entropia(pi) for pi in probabilidades])
print(f'La entropia total es {round(entropia_total,3)}')

Podemos ver que este valor coincide con el que declara el método `DecisionTreeClassifier` de `scikit-learn` en el árbol de decisión que obtuvimos anteriormente.


Ahora, **¿Por qué eligió la variable pelo para ramificar? ¿Cómo lo hizo?**

Para responder esto, calculemos la ganancia de entropía para cada variable descriptiva. Esto lo podemos pensar como:

$GananciaEntropia(variable_d) = Entropia_{total} - Entropia(variable_d)$

Lo que se puede escribir como:

$GananciaEntropia(variable_d) = Entropia_{total} - \sum_{t \in variable_d} P(variable_d=t) * H(variable_d=t) $

que es igual a:

$GananciaEntropia(variable_d) = Entropia_{total} - \sum_{t \in variable_d} P(variable_d=t) * (-\sum_{k \in target} P(target=k \cap variable_d = t)) * \log_2(P((target=k \cap variable_d = t)))$

In [ ]:
data[data.Dientes == 1]

In [ ]:
data[data.Dientes == 0]

In [ ]:
#Probabilidades de ser mamífero o reptil dado que tiene dientes
P_especie_dientes = calc_prob(data[data.Dientes == 1]['Especie'])
#Probabilidades de ser mamífero o reptil dado que no tiene dientes
P_especie_nodientes = calc_prob(data[data.Dientes == 0]['Especie'])
#Probabilidad de tener dientes
P_dientes = calc_prob(data['Dientes'])

#entropias
entropia_dientes = P_dientes[1] * (entropia(P_especie_dientes[0]) + entropia(P_especie_dientes[1])) \
                +  P_dientes[0] * (entropia(P_especie_nodientes[0]) + entropia(P_especie_nodientes[1]))

entropia_dientes

In [ ]:
#información ganada
entropia_total - entropia_dientes 

In [ ]:
def ganancia_de_info(var,data=data,respuesta='Especie'):
    '''
    Función que calcula la ganancia de información utilizando la entropía como medida
    de información.
    
    variables de entrada:
    var (str): nombre de la variable sobre la cuál vamos a calcular la ganancia de la información.
    data (dataFrame): el conjunto de datos de donde sacar la varianza
    respuesta (str): nombre de la variable respuesta
    
    Devuelve la ganancia de información (float)
    '''
    probabilidades = calc_prob(data[respuesta])
    entropia_total = np.sum([entropia(pi) for pi in probabilidades])
    P_especie_var = calc_prob(data[respuesta][data[var] == 1])
    P_especie_novar = calc_prob(data[respuesta][data[var] == 0])
    P_var = calc_prob(data[var])
    
    entropia_var = P_var[1] * np.sum([entropia(pi) for pi in P_especie_var]) \
                    +  P_var[0] * np.sum([entropia(pi) for pi in P_especie_novar])
    return entropia_total - entropia_var

print(f'ganancia de información variable dientes: {ganancia_de_info("Dientes"):.3f}')
print(f'ganancia de información variable pelo: {ganancia_de_info("Pelo"):.3f}')
print(f'ganancia de información variable respira: {ganancia_de_info("Respira"):.3f}')
print(f'ganancia de información variable piernas: {ganancia_de_info("Piernas"):.3f}')

In [ ]:
#recordemos que las variables son ["Dientes","Pelo","Respira","Piernas"]
animal_raro_1 = [0,0,0,0]
animal_raro_2 = [1,1,0,0]
model.predict([animal_raro_1,animal_raro_2])

# Caso un poco más complejo

In [ ]:
#cargamos los datos
iris = datasets.load_iris()
X, y = iris.data, iris.target

#armamos dataframe para visualizar
df = pd.DataFrame(X,columns=iris.feature_names)
df['Especie'] = y
df['Especie'].replace({0:'setosa', 1:'versicolor', 2:'virginica'},
                      inplace=True)

print(iris.target_names)
print(iris.feature_names)
df

### Regiones de decisión

Para facilitar la comprensión, lo que vamos a hacer es sólo generar un modelo de clasificación teniendo en cuenta únicamente las características del sépalo. Como son dos variables, podremos ver exactamente las regiones de decisión en un gráfico bidimensional (sin tener que fijar las otras variables).

In [ ]:
tree.DecisionTreeClassifier?

In [ ]:
X1 = X[:,:2]
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=7,
                                   random_state=1)
clf.fit(X1, y)

print("The prediction accuracy is: ",clf.score(X1,y)*100,"%")
# Busco los valores máximos y mínimos de las variables de sépalo        
x_min, x_max = iris.data[:, 0].min() - 1, iris.data[:, 0].max() + 1
y_min, y_max = iris.data[:, 1].min() - 1, iris.data[:, 1].max() + 1

# Armo grilla de puntos donde vamos a predecir para armar las regiones
xx1, xx2 = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))
Z2 = clf.predict(np.c_[xx1.ravel(),xx2.ravel()])
Z2 = Z2.reshape(xx1.shape)

plt.figure(figsize=(10,15))
plt.subplot(2,1,1)
cmap = ListedColormap(["orange","mediumspringgreen", "darkviolet"])
plt.contourf(xx1, xx2, Z2, alpha=0.4, cmap=cmap, levels=3)

#defino los colores necesarios para que quede más bonito y consistente con los colores del árbol.
cmap = ListedColormap(["mediumspringgreen", "darkviolet"])
ycolor = []
for caso in y:
    if caso == 0:
        ycolor.append("orange")
    elif caso == 1:
        ycolor.append("mediumspringgreen")
    elif caso == 2:
        ycolor.append("darkviolet")
    else:
        ycolor.append(np.nan)
        
#Agregamos todos los otros datos, pero más suavecitos. El valor de alpha determina la transparencia.
plt.scatter(df['sepal length (cm)'],
            df['sepal width (cm)'], c=ycolor,alpha=0.7)
#Agregamos formato al primer gráfico
plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')


plt.subplot(2,1,2)
tree.plot_tree(clf,
               feature_names=iris.feature_names[:2],
               filled=True)
plt.show()

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=iris.feature_names[:2],  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)
graph = graphviz.Source(dot_data)
graph

Otra forma de inspeccionar el árbol es el método `export_text` del módulo `tree` de `sklearn` que permite visualizar el árbol de decisión en texto. A partir de inspeccionar el árbol de decisión, podemos entender la forma de las regiones de decisión.

In [ ]:
print(tree.export_text(clf,feature_names=iris.feature_names[:2]))

## Regression Trees

En este caso el criterio de regresión suele ser el ECM

$H(Q_m)=\frac{1}{n_m}\sum_{y\in Q_m}(y-\bar{y}_m)^2$

[Documentación](https://scikit-learn.org/stable/modules/tree.html#tree-mathematical-formulation)

In [ ]:
from sklearn.datasets import fetch_california_housing
X_california, y_california = fetch_california_housing(return_X_y=True, as_frame=True)
california = fetch_california_housing()

In [ ]:
print(california['DESCR'])  # descripción del dataset

In [ ]:
X2 = california.data[:,0] # Usamos la feature MedInc
y2 = california.target

# Sort X and y by ascending values of X

sort_idx = X2.flatten().argsort()
X2 = X2[sort_idx].reshape(-1, 1)
y2 = y2[sort_idx]

In [ ]:
tree.DecisionTreeRegressor?

In [ ]:
clf2 = tree.DecisionTreeRegressor(max_depth=3, criterion="squared_error")
#clf2 = tree.DecisionTreeRegressor(criterion="squared_error")

clf2 = clf2.fit(X2, y2)

### Qué nos están dando las hojas del árbol en este caso?

In [ ]:
dot_data = tree.export_graphviz(clf2, out_file=None, 
                         filled=True, rounded=True,  
                         special_characters=True)
graph = graphviz.Source(dot_data)
graph

### Grafiquemos

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(X2, y2, c='steelblue',
            edgecolor='white', s=70)
plt.plot(X2, clf2.predict(X2),
         color='black', lw=2)
plt.xlabel('median income in block group [MedInc]')
plt.ylabel('Median House Values $1000s [MedHV]')
plt.show()